In [2]:
from google.cloud import aiplatform

In [5]:
!gsutil mb -l europe-west1 gs://vertex_e2e_example

Creating gs://vertex_e2e_example/...


In [7]:
bucket = "gs://vertex_e2e_example"

In [11]:
print(bucket)

gs://vertex_e2e_example


In [14]:

job = aiplatform.CustomTrainingJob(
    #display_name=JOB_NAME,
    display_name="first model on vertex",
    script_path="train.py",
    #container_uri=TRAIN_IMAGE,
    container_uri="europe-docker.pkg.dev/vertex-ai/training/tf-cpu.2-6:latest",
    #requirements=["google-cloud-bigquery>=2.20.0"],
    #model_serving_container_image_uri=DEPLOY_IMAGE,
    staging_bucket=bucket,
    project='vf-grp-commercial-tst-explore',
    location='europe-west1'
)

In [15]:
model = job.run(
       # dataset=dataset,
       # model_display_name=MODEL_DISPLAY_NAME,
       # bigquery_destination=f"bq://{PROJECT_ID}",
       # args=CMDARGS,
        replica_count=1,
       # machine_type=TRAIN_COMPUTE,
    machine_type='n1-standard-4',
        accelerator_count=0,
    )

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://vertex_e2e_example/aiplatform-2021-11-01-15:25:53.281-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://vertex_e2e_example/aiplatform-custom-training-2021-11-01-15:25:53.493 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/7498581340502097920?project=387138108602
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west1/training/616870003647447040?project=387138108602
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/387138108602/locations/europe-west1/trainingPipelines/7498581340502097920 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/387138108602/locations/europe-west1/trainingPipe

RuntimeError: Training failed with:
code: 3
message: "The replica workerpool0-0 exited with a non-zero status of 1. \nTraceback (most recent call last):\n  File \"/opt/conda/lib/python3.7/runpy.py\", line 193, in _run_module_as_main\n    \"__main__\", mod_spec)\n  File \"/opt/conda/lib/python3.7/runpy.py\", line 85, in _run_code\n    exec(code, run_globals)\n  File \"/root/.local/lib/python3.7/site-packages/aiplatform_custom_trainer_script/task.py\", line 113, in <module>\n    mode=\'train\')\n  File \"/root/.local/lib/python3.7/site-packages/aiplatform_custom_trainer_script/task.py\", line 57, in create_dataset\n    pattern, batch_size, CSV_COLUMNS, DEFAULTS)\n  File \"/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/experimental/ops/readers.py\", line 479, in make_csv_dataset_v2\n    filenames = _get_file_names(file_pattern, False)\n  File \"/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/experimental/ops/readers.py\", line 1124, in _get_file_names\n    rai

#### above fails because the data was being picked up from a local file system; however for vertex ai training the data needs to be in gcs or bq location

https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/art_and_science_of_ml/solutions/export_data_from_bq_to_gcs.ipynb

%%writefile --append train.py

tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])